Import pandas and loading the datasets into dataframes.

In [5]:
import pandas as pd

purchase_behavior_df = pd.read_csv('/content/purchase_behaviour.csv')
transaction_data_df = pd.read_csv('/content/transaction_data.csv')


Mergeing the 2 datasets on loyalty card number for calculations.

In [ ]:
merged = pd.merge(transaction_data_df, purchase_behavior_df, on='LYLTY_CARD_NBR', how='inner')

Checking for null values.

In [6]:
merged.isnull().sum()

,0
DATE,0
STORE_NBR,0
LYLTY_CARD_NBR,0
TXN_ID,0
PROD_NBR,0
PROD_NAME,0
PROD_QTY,0
TOT_SALES,0
LIFESTAGE,0
PREMIUM_CUSTOMER,0


The merged dataframe is grouped based on product name and number.
Total quantity sold per product is aggregated into 'sold'.
The total money generated per product is aggregated into 'sales'.
The result is then sorted in ascending order to find the top products.

In [11]:
product_sales_df = merged.groupby(['PROD_NAME', 'PROD_NBR']).agg(
    sold = ('PROD_QTY', 'sum'),
    sales = ('TOT_SALES', 'sum')
  ).reset_index()
top_prod = product_sales_df.sort_values(by = 'sales', ascending=False)

The top 5 products are as follows:

In [12]:
top_prod.head()

,PROD_NAME,PROD_NBR,sold,sales
11,Dorito Corn Chp Supreme 380g,4,6509,40352.0
86,Smiths Crnkle Chip Orgnl Big Bag 380g,14,6164,36367.6
77,Smiths Crinkle Chips Salt & Vinegar 330g,16,6106,34804.2
33,Kettle Mozzarella Basil & Pesto 175g,102,6381,34457.4
76,Smiths Crinkle Original 330g,7,6018,34302.6


The merged dataframe is grouped based on loyalty card number to group each customer. The 'spent' variable stores the value of total money spent by the customer.
The total_buys variable stores the number of uneque transactions by a customer.

In [17]:
customer_df = merged.groupby(['LYLTY_CARD_NBR']).agg(
    spent = ('TOT_SALES', 'sum'),
    total_buys = ('TXN_ID', 'nunique')
).reset_index()

To get the most loyal customers, we can take the intersection between the most spending and the customers with the most uneque purchases.

In [31]:
top_buyers = customer_df.sort_values(by='total_buys', ascending=False)
top_buyers = top_buyers.head(100)

In [32]:
top_spenders = customer_df.sort_values(by='spent', ascending=False)
top_spenders = top_spenders.head(100)

In [33]:
loyal_customers = pd.merge(top_spenders, top_buyers, on='LYLTY_CARD_NBR', how='inner', suffixes=('_spent', '_buys'))

getting characeristics of the loyal customers.

In [34]:
loyal_customers = pd.merge(
    loyal_customers, purchase_behavior_df[['LYLTY_CARD_NBR', 'LIFESTAGE', 'PREMIUM_CUSTOMER']],
    on='LYLTY_CARD_NBR',
    how='left'
)

Top 10 loyal customers and their characteristics

In [35]:
loyal_customers.head(10)

,LYLTY_CARD_NBR,spent_spent,total_buys_spent,spent_buys,total_buys_buys,LIFESTAGE,PREMIUM_CUSTOMER
0,230078,138.60,17,138.60,17,OLDER FAMILIES,Budget
1,63197,132.80,15,132.80,15,OLDER FAMILIES,Budget
2,259009,127.20,13,127.20,13,OLDER SINGLES/COUPLES,Mainstream
3,162039,126.80,18,126.80,18,OLDER FAMILIES,Mainstream
4,230154,124.40,14,124.40,14,OLDER FAMILIES,Budget
5,94185,122.80,16,122.80,16,YOUNG FAMILIES,Premium
6,130090,122.65,14,122.65,14,YOUNG FAMILIES,Budget
7,179228,120.80,16,120.80,16,YOUNG FAMILIES,Budget
8,172032,120.10,16,120.10,16,YOUNG SINGLES/COUPLES,Mainstream
9,116181,117.60,16,117.60,16,YOUNG FAMILIES,Premium
